In [45]:
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge,BatchNormalization,Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Embedding,LSTM,GRU,TimeDistributed,RepeatVector,Merge,Input,merge,UpSampling2D
from keras.preprocessing import sequence
from keras import callbacks
from keras.optimizers import SGD, RMSprop, Adam

import numpy as np
from vgg16 import Vgg16
import matplotlib.pyplot as plt
import PIL.Image

from tqdm import tqdm

import utils as utils

import cPickle as pickle
import string

import collections
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

import re
from numpy.random import random, permutation, randn, normal 

import os

import preprocessing as preproc

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from matplotlib import animation
from IPython.display import display, HTML

import pandas as pd


In [2]:
def search_images_by(searched_word,images,predicted_captions):
    lmtzr = WordNetLemmatizer()
    lemm_word = lmtzr.lemmatize(searched_word)
    
    found_indexes = []
    for index,caption in enumerate(predicted_captions):
        lemm_caption_words = [lmtzr.lemmatize(word) for word in caption.split()]
        if lemm_word in lemm_caption_words:
            found_indexes.append(index)
    
    return ([images[i] for i in found_indexes],[predicted_captions[i] for i in found_indexes])

    
# def make_prediction(random_number,images_concat_t,vgg_model):
#     startIndex = word2index["START"]
#     start_captions = [[startIndex]]
#     start_captions = sequence.pad_sequences(start_captions, maxlen=MAX_CAPTION_LEN,padding='post')

#     firstImage = np.expand_dims(images_concat_t[random_number], axis=0)
#     first_image_vgg_features = vgg_model.predict(firstImage)
#     first_image_input = np.squeeze(first_image_vgg_features)[0].reshape(1,4096)
    
#     firstCaption = np.expand_dims(start_captions[0], axis=0) 
#     prev_word_indexed_captions = np.expand_dims(list(start_captions[0]), axis=0)
    
#     outputs = []

#     endGenerated = False
#     i = 0
        
#     while ((not endGenerated) & (i < MAX_CAPTION_LEN-1)):
       
#         predictions = model.predict([first_image_input, firstCaption, prev_word_indexed_captions])
#         predictions = predictions[0]
        
#         currentPred = predictions[i]
        
#         max_index = np.argmax(currentPred)
        
#         outputs.append(max_index)
#         firstCaption[0,i+1] = max_index
        
#         prev_word_indexed_captions[0,i+1] = firstCaption[0,i]
                
#         i+=1

#         if(index2word[max_index] == "END"):
#             endGenerated = True

#     caption = ' '.join([index2word[x] for x in firstCaption[0][1:i]])

#     drawImage = firstImage[0]
#     drawImageT = np.transpose(drawImage,(1,2,0))
#     plt.imshow(drawImageT)
    
#     return (drawImageT,caption)

# def make_prediction_on_dataset(images_concat_t, window_start = None, no_images = None):
    
#     if(window_start == None):
#         window_start = 0
        
#     if(no_images == None):
#         no_images = len(images_concat_t)
    
#     vgg_model = get_vgg_model()
    
#     images2Captions = [make_prediction(i,images_concat_t,vgg_model) for i in tqdm(range(window_start,window_start+no_images))]
#     images = [image2Caption[0] for image2Caption in images2Captions]
#     predicted_captions = [image2Caption[1] for image2Caption in images2Captions]

    
#     return (images,predicted_captions)
        

In [3]:
base_path = data_path+"app-100-length-15/"

train_path = base_path + train_folder
val_path = base_path + val_folder

NR_TRAIN_EXAMPLES = 219136
NR_TEST_EXAMPLES = 28858
SMALL_NR_TEST_EXAMPLES = 1000

# Read Serialized Data - Images

In [5]:
# test_images_concat_t = preproc.get_images_concat(
#     val_path + images_concat_folder+ 'images_concat_0.bc',
#     SMALL_NR_TEST_EXAMPLES)

# print(test_images_concat_t.shape)

# Load precomputed misc data structures

In [6]:
train_img_vgg_path = base_path + train_folder + batch_folder + images_vgg_4096_folder
train_indexed_captions_path = base_path + train_folder + batch_folder + indexed_captions_folder
train_raw_captions_path = base_path + train_folder + batch_folder + captions_folder
train_future_words_path = base_path + train_folder + batch_folder + indexed_future_words_folder

test_img_vgg_path = base_path + val_folder + batch_folder +images_vgg_4096_folder
test_indexed_captions_path = base_path + val_folder + batch_folder + indexed_captions_folder
test_raw_captions_path = base_path + val_folder + batch_folder+captions_folder
test_future_words_path = base_path + val_folder + batch_folder+indexed_future_words_folder

indexed_prev_captions_folder = "indexed-prev-captions/"
train_prev_captions_path = base_path + train_folder + batch_folder + indexed_prev_captions_folder
test_prev_captions_path = base_path + val_folder + batch_folder + indexed_prev_captions_folder



In [51]:
reload(utils)

<module 'utils' from 'utils.py'>

In [52]:
# unique_words = preproc.load_obj(base_path + general_datastruct_folder+"unique_words")
# word2index = preproc.load_obj(base_path+general_datastruct_folder+"word2index")
# index2word = preproc.load_obj(base_path+general_datastruct_folder+"index2word")

(unique_words, word2index, index2word) = utils.load_language_data_structures(base_path + general_datastruct_folder)

(train_captions_raw,_) = preproc.get_captions_raw_and_indexed(train_raw_captions_path,train_indexed_captions_path)
(test_captions_raw,_) = preproc.get_captions_raw_and_indexed(test_raw_captions_path,test_indexed_captions_path)


100%|██████████| 11/11 [00:00<00:00, 677.88it/s]


In [8]:
VOCAB_SIZE = len(unique_words)
MAX_CAPTION_LEN = 15 # ATENTIE AICI

print("MAX_CAPTION_LEN = %s"%MAX_CAPTION_LEN)
print("VOCAB_SIZE = %s"%VOCAB_SIZE)

# Model Building

In [43]:
emb = nnModel.get_embeddings(index2word, VOCAB_SIZE, EMB_SIZE)
model = nnModel.build_model(emb,MAX_CAPTION_LEN, VOCAB_SIZE, EMB_SIZE)

Found = 840
Not found = 8


In [44]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
repeatvector_9 (RepeatVector)    (None, 15, 4096)      0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 15, 300)       254700                                       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 15, 300)       1200                                         
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 15, 300)       254700                                       
___________________________________________________________________________________________

# Training the model

In [57]:
history = model.fit_generator(
                    nnModel.generate_arrays_from_file(train_img_vgg_path,train_indexed_captions_path,
                                              train_future_words_path,train_prev_captions_path),
                    samples_per_epoch = 219136,
                    nb_epoch=1,
                    validation_data = nnModel.generate_arrays_from_file(test_img_vgg_path,test_indexed_captions_path,
                                                   test_future_words_path,test_prev_captions_path),
                    nb_val_samples = 2048,
                    callbacks=[#callbacks.RemoteMonitor(),
                               callbacks.CSVLogger("./training.log")]
                   )

In [ ]:
model.evaluate_generator(nnModel.generate_arrays_from_file(test_img_vgg_path,test_indexed_captions_path,
                                                   test_future_words_path,test_prev_captions_path),
                         val_samples = 2048)

In [ ]:
# model.save_weights(save_path + models_folder+"big/" +'app_100_length_15_past_word_30_epoch_300d_gru_2x1048_big.h5')

In [56]:
model.load_weights(save_path + models_folder+"big/" +'app_100_length_15_past_word_20_epoch_300d_gru_2x1048_big.h5')

# Evaluate the model

In [ ]:
model.evaluate_generator(generate_arrays_from_file(train_img_vgg_path,
                                                   train_indexed_captions_path,
                                                   train_future_words_path,
                                                   train_prev_captions_path),
                        val_samples = NR_TRAIN_EXAMPLES,
                        )

In [ ]:
model.evaluate_generator(generate_arrays_from_file(test_img_vgg_path,
                                                   test_indexed_captions_path,
                                                   test_future_words_path,
                                                   test_prev_captions_path),
                        val_samples = NR_TEST_EXAMPLES)

# Testing the model

In [ ]:
def strip_caption_list(captions_raw):
    
    stripped_caption_list = []
    caption_list = list(captions_raw)
    for caption in caption_list:
        stripped = str(caption).split(" ")[1:-1]
        stripped_caption_list.append(" ".join(stripped))
        
    return stripped_caption_list

In [ ]:
window_start = 500
nr_images = 200

# images_concat_t = train_images_concat_t #ATENTIE
# real_captions = train_captions_raw

images_concat_t = test_images_concat_t
real_captions = strip_caption_list(test_captions_raw)

print(images_concat_t.shape)

(images,predicted_captions) = make_prediction_on_dataset(images_concat_t,window_start,nr_images)


In [ ]:
# merged_captions = [str(a) +" ------ " + str(b)  for a,b in zip(predicted_captions,real_captions)]
# merged_captions[:10]

In [ ]:
preproc.plot_predictions(images,titles = predicted_captions)

In [ ]:
common_words2app = preproc.most_common_words(predicted_captions,500)
common_words2app

In [ ]:
searched_word = "bus"
(found_images,found_captions) = search_images_by(searched_word,images,predicted_captions)
print("Number of results = %d"%len(found_images))

In [ ]:
preproc.plot_predictions(found_images,found_captions)

# Semantic Segmentation

In [ ]:
def get_unet(img_rows, img_cols,nr_labels):
    
    inputs = Input((3, img_rows, img_cols))
    bn = BatchNormalization()(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(bn)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(nr_labels, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [ ]:
cat_2_supercat_dict = load_obj("/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/detection/misc/cat_2_supercat_dict.pkl")
supercat2indexpath = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/detection/misc/cat_2_index_dict_person_animal_vehicle_10e.pkl"
supercat_2_index_dict = load_obj(supercat2indexpath)

In [ ]:
index_2_supercat_dict = dict([(vv,kk) for kk,vv in supercat_2_index_dict.iteritems()])

print(supercat_2_index_dict)
print(index_2_supercat_dict)

In [ ]:
def can_generate_segmentation(caption):
    caption_words = caption.split(" ")
    match_word_arr = [word in cat_2_supercat_dict for word in caption_words]
    found = np.any(match_word_arr)
    
    if(not found):
        return (False,-1)
    
    word_categories = np.array(caption_words)[np.array(match_word_arr)]
                                           
    if(len(word_categories) == 0):
        return (False,-1)
    
    category = word_categories[0]
   
    if(category not in cat_2_supercat_dict):
        return (False,-1)

    supercategory = cat_2_supercat_dict[category]

    if(supercategory not in supercat_2_index_dict):
        return (False,-1)
        
    return (True,supercat_2_index_dict[supercategory])

In [ ]:
seg_net_model = get_unet(224,224,nr_labels = 3)
save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/detection/models/"
seg_net_model.load_weights(save_path+"person_animal_vehicle_10e.h5")

In [ ]:
seg_mask_to_supercat_list = [can_generate_segmentation(caption) for caption in predicted_captions]
seg_mask = [mask for (mask,_) in seg_mask_to_supercat_list]

In [ ]:
seg_captions = np.asarray(predicted_captions)[np.asarray(seg_mask)]
seg_images = np.asarray(images)[np.asarray(seg_mask)]
image_supercat_list = [supercat for (_,supercat) in np.asarray(seg_mask_to_supercat_list)[np.asarray(seg_mask)]]

In [ ]:
for i in range(5):
    print(seg_captions[i])
    print("\t ==>" + index_2_supercat_dict[image_supercat_list[i]])
    print("---")

In [ ]:
seg_images_np = np.stack(seg_images)
transposed_img = np.transpose(seg_images_np,(0,3,1,2))
transposed_img.shape

In [ ]:
predictions = seg_net_model.predict(transposed_img)
predictions.shape

In [ ]:
category_predictions = []
for index in range(len(predictions)):
    s_index = image_supercat_list[index]
    current_prediction = predictions[index]
    category_predictions.append(current_prediction[s_index:s_index+1,:,:])

In [ ]:
def plotImageCaptionToMask(images, predictions, titles):
  
    nr_rows = len(predictions)
    
    figsize = 8
    plt.figure(figsize=(figsize,figsize))
    for index in range(nr_rows):
   
        squeezed_pred = np.squeeze(predictions[index])
        
        image = images[index]
        
        s_index = image_supercat_list[index]
        supercategory = index_2_supercat_dict[s_index]
        
        
        nr_cols = 2
        
        plt.subplot(1,nr_cols,1)
        plt.title(titles[index] + " => " + supercategory)
        plt.imshow(image)

        
        
        plt.subplot(1,nr_cols,2)

        plt.imshow(squeezed_pred)
        plt.imshow(image, alpha=0.5)
    
        
        plt.figure(figsize=(figsize,figsize))
        
    plt.show()

In [ ]:
plotImageCaptionToMask(seg_images, category_predictions,seg_captions)

# Video

In [ ]:
import skvideo.io



In [ ]:
predictions = seg_net_model.predict(video_frames)
predictions.shape

In [ ]:
animal_index = supercat_2_index_dict['person']
animal_index

In [ ]:
animal_preds = predictions[:,animal_index:animal_index+1,:,:]
animal_preds.shape

In [ ]:
transposed_animal_preds = np.squeeze(animal_preds)
transposed_animal_preds.shape

In [ ]:
# plot_movie_mp4(video_frames)

In [ ]:
plot_movie_mp4(transposed_animal_preds)

## Video Search

In [ ]:

def get_concat_video(video_paths):
    list_of_video_frames = [ get_mp4_vid_frames(video_base_path + video_path + ".mp4") for video_path in video_paths]
    video_frame_list = [y for x in list_of_video_frames for y in x]
    input_video_frames = np.transpose(np.asarray(video_frame_list),(0,3,1,2))
    return input_video_frames


def plot_index_arr(video_frames,found_indexes):
    index_arr = get_index_arr(video_frames,found_indexes)
    plt.figure()
    ind = np.asarray(range(len(index_arr)))
    plt.bar(ind, index_arr)
    plt.show()

In [ ]:
video_base_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/video/"
video_paths = ["horse","boat","baseball"]

In [ ]:
input_video_frames = get_concat_video(video_paths)
print(input_video_frames.shape)

(video_frames,video_frames_captions) = make_prediction_on_dataset(input_video_frames,0,len(input_video_frames))

## Search by word

## See most common objects in video

In [ ]:
common_words2app = preproc.most_common_words(video_frames_captions,50)
common_words2app

In [ ]:
most_common_words = [word for (word,count) in common_words2app[:4]]
most_common_words

In [ ]:
def get_index_arr(video_frames,found_indexes):
    index_arr = np.zeros(len(video_frames))
    for found_index in found_indexes:
        index_arr[found_index] = 1
        
    return index_arr
    
def get_found_index_arr(video_search_word,video_frames,video_frames_captions):
    (_,_,found_indexes) = search_video_by(video_search_word,video_frames,video_frames_captions)
    return get_index_arr(video_frames,found_indexes)
    return found_indexes

def plot_bar_chart(word_indexes_arr,most_common_words):
    index = range(len(word_indexes_arr[0]))

    plt.figure()
    plt.bar(index, word_indexes_arr[0],
                     color='r',
                     label=most_common_words[0])

    plt.bar(index, word_indexes_arr[1],
                     color='b',
                     label=most_common_words[1])

    plt.bar(index, word_indexes_arr[2],
                     color='g',
                     label=most_common_words[2])

    plt.bar(index, word_indexes_arr[3],
                     color='y',
                     label=most_common_words[3])

    plt.legend()
    plt.show()

    
    
def plot_frame_occurences(most_common_words,video_frames,video_frames_captions):
    word_indexes_arr = [get_found_index_arr(word,video_frames,video_frames_captions) for word in most_common_words]
    plot_bar_chart(word_indexes_arr,most_common_words)

In [ ]:
plot_frame_occurences(most_common_words,video_frames,video_frames_captions)

# Make predictions on misc dataset

In [ ]:
misc_images_path = save_path + misc_images_folder

In [ ]:
misc_images = []
for img_path in os.listdir(misc_images_path):
    img = PIL.Image.open(misc_images_path+img_path)
    img = img.resize((224, 224), PIL.Image.NEAREST)
    img = np.asarray(img)
    img = np.transpose(img,(2,0,1))
    img = np.expand_dims(img,axis=0)
    
    misc_images.append(img)
    
stacked_images = np.vstack(misc_images)

In [ ]:
(misc_images,misc_predicted_captions) = make_prediction_on_dataset(stacked_images)
preproc.plot_predictions(misc_images,misc_predicted_captions)

## Prepare data for "Caption Turing Test"

In [ ]:
def get_all_captions(image_data_arr):
    caption_bucket_list = preproc.get_captions_list(image_data_arr)
    caption_bucket_length = len(caption_bucket_list[0])
    
    captions = np.stack(caption_bucket_list)
    
    print("caption_bucket_length = %d"%caption_bucket_length)
    print("captions.shape = %s"%str(captions.shape))
    
    return (caption_bucket_list)


def get_imgs_and_captions(base_images_path,base_annotation_path,nr_items):
    image_data_arr = preproc.get_image_data_arr(base_images_path,base_annotation_path,nr_items)
    
    captions= get_all_captions(image_data_arr)
    images = preproc.construct_images_concat_t(image_data_arr)
    
    print("images.shape = %s"%str(images.shape))
    return (images,captions)
    
    

In [ ]:
base_images_path = val_images_path
base_annotation_path = val_annotation_path

(images,caption_bucket_list) = get_imgs_and_captions(base_images_path,base_annotation_path,100)

In [ ]:
(_,predicted_captions) = make_prediction_on_dataset(images,0,99)

In [ ]:
real_captions_1 = strip_caption_list(caption_bucket_list[0])
real_captions_2 = strip_caption_list(caption_bucket_list[1])

In [ ]:
turing_df = percentile_list = pd.DataFrame(
    {'image': list(images),
     'predicted_caption': predicted_captions,
     'real_caption_1': real_captions_1,
     'real_caption_2': real_captions_2
    })

In [ ]:
turing_df.head()